In [3]:
import xarray as xr

ml_wind = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2",
    # chunks={"time": 48},
    # decode_times=False,
    consolidated=True,
)

ml_wind = ml_wind[["u_component_of_wind", "v_component_of_wind", "temperature", "vertical_velocity"]]

print(f"Model wind dataset size {ml_wind.nbytes / (1024 ** 4):.2f} TiB")
ml_wind

Model wind dataset size 308.72 TiB


<xarray.Dataset>
Dimensions:              (time: 552264, level: 37, latitude: 721,
                          longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 1959-01-01 ... 2021-12-31T23:0...
Data variables:
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [8]:
wind_slice = ml_wind.sel(level=slice(150, 1000), time=slice("2012", "2022"))
# wind_slice = wind_slice.astype("float16")
wind_slice = wind_slice.convert_calendar("noleap")
wind_slice = wind_slice.groupby("time.dayofyear")
wind_slice = wind_slice.mean()

print(f"Wind slice size {wind_slice.nbytes / (1024 ** 3):.02f} GiB")
wind_slice

Wind slice size 141.17 GiB


<xarray.Dataset>
Dimensions:              (dayofyear: 365, level: 25, latitude: 721,
                          longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 150 175 200 225 250 ... 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * dayofyear            (dayofyear) int64 1 2 3 4 5 6 ... 361 362 363 364 365
Data variables:
    u_component_of_wind  (dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(1, 25, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(1, 25, 721, 1440), meta=np.ndarray>
    temperature          (dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(1, 25, 721, 1440), meta=np.ndarray>
    vertical_velocity    (dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(1, 25, 721, 1440), meta=np.ndarray>

In [9]:
wind_chunk = wind_slice.sel(dayofyear=1)
print(f"Wind chunk size {wind_chunk.nbytes / (1024 ** 2):.02f} MiB")

Wind chunk size 396.07 MiB


In [10]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    wind_chunk.to_netcdf()

[##                                      ] | 6% Completed | 14m 52sss


KeyboardInterrupt: 

In [4]:
import metview as mv


# Needs installation of metview binary

def attribute_fix(ds):
    """Needed to fix a low-level bug in ecCodes.
    
    Sometimes, shortNames get overloaded in ecCodes's table. 
    To eliminate ambiguity in their string matching, we
    force ecCodes to make use of the paramId, which is a
    consistent source-of-truth.
    """
    for var in ds:
        attrs = ds[var].attrs
        attrs.pop("GRIB_cfName", None)
        attrs.pop("GRIB_cfVarName", None)
        attrs.pop("GRIB_shortName", None)
        ds[var].attrs.update(attrs)
    return ds

In [5]:
wind_fieldset = mv.dataset_to_fieldset(attribute_fix(wind_slice).squeeze())

/Users/bhavyemathur/DataspellProjects/wind-prediction-merra2/venv/lib/python3.11/site-packages/cfgrib/xarray_to_grib.py:261: FutureWarning: GRIB write support is experimental, DO NOT RELY ON IT!
  warnings.warn("GRIB write support is experimental, DO NOT RELY ON IT!", FutureWarning)


In [6]:
from scipy.ndimage import shift

uv_wind_np = wind_fieldset.select(level=1000).values()
uv_wind_np = uv_wind_np[1].reshape(721, 1440)
uv_wind_np = uv_wind_np[::-1]
uv_wind_np = shift(uv_wind_np, (0, 720), mode="wrap")
uv_wind_np.shape

NameError: name 'uv_wind' is not defined

In [5]:
import modules.plot as plot
import matplotlib.pyplot as plt

fig, ax, mesh, plot_kwargs = plot._setup_latitude_longitude_contour(uv_wind_np, "", [], "image")

plot._update_latitude_longitude_contour(mesh, uv_wind_np, ax, "image")
plt.show()

NameError: name 'uv_wind_np' is not defined